In [2]:
import pandas as pd
import geopandas as gpd
import numpy as np
import censusdata
import warnings; warnings.simplefilter('ignore')
import math

In [3]:
#censusdata.search('acs5', 2015, 'concept', 'median income')

In [4]:
censusdata.printtable(censusdata.censustable('acs5', 2015, 'B19301'))

Variable     | Table                          | Label                                                    | Type 
-------------------------------------------------------------------------------------------------------------------
B19301_001E  | B19301. Per Capita Income in t | Per capita income in the past 12 months (in 2015 Inflati | int  
-------------------------------------------------------------------------------------------------------------------


In [24]:
df = censusdata.download('acs5', 2015, censusdata.censusgeo([('state', '25'), ('county', '025'), ('tract', '*')]),
                             ['B19013_001E', 'B19301_001E'])
df = df.reset_index(drop=False)
df.rename(columns = {'index' : 'TRACT', 'B19013_001E' : 'Median Household Income', 
                     'B19301_001E' : 'Per Capita Income'}, inplace = True)
for x in range(len(df)):
    df.loc[x, 'TRACT'] = df['TRACT'].unique()[x].geo[2][1]
    
df = df.astype({"TRACT": int})

df = df.replace({-666666666.0: None}, regex=True)
df

,TRACT,Median Household Income,Per Capita Income
0,20303,81607.0,55239.0
1,40401,53438.0,52650.0
2,40801,60192.0,57607.0
3,50101,42875.0,20550.0
4,50901,60769.0,24438.0
...,...,...,...
199,101101,59133.0,18857.0
200,130300,108472.0,50684.0
201,140102,65457.0,30841.0
202,140300,44771.0,21049.0


In [82]:
df.to_csv('test.csv')

In [25]:
template = pd.read_csv('Precinct_Make_Up_Of_Census_Tracts.csv')
template = template.drop('Unnamed: 0', axis=1)

income = pd.DataFrame(0, index=np.arange(len(template.columns[1:])),
                        columns=df.columns[1:])

income.insert(0, 'WARD_PRECI', template.columns[1:])

for i in range(len(income)):
    prct = income['WARD_PRECI'].loc[i]
    temp = template[['TRACT', prct]]
    tracker = []
    for n in range(len(temp)):
        if temp[prct].loc[n] != 0:
            tracker.append(temp['TRACT'].loc[n])
    med = [float(df[df['TRACT'] == x]['Median Household Income']) for x in tracker if math.isnan(float(df[df['TRACT'] == x]['Median Household Income'])) == False]
    per_c = [float(df[df['TRACT'] == x]['Per Capita Income']) for x in tracker if math.isnan(float(df[df['TRACT'] == x]['Per Capita Income'])) == False]
    try:
        income.loc[i, 'Median Household Income'] = round(sum(med)/len(med))
    except ZeroDivisionError:
        income.loc[i, 'Median Household Income'] = 0
    try:
        income.loc[i, 'Per Capita Income'] = round(sum(per_c)/len(per_c))
    except ZeroDivisionError:
        income.loc[i, 'Per Capita Income'] = 0
        
income.to_csv('Income_Metrics_By_Precinct_2011-2015.csv')
        
neighborhoods = gpd.read_file('Boston_Neighborhoods.shp')

props = pd.read_csv('Neighborhood_Make_Up_Of_Precincts.csv')

inc = pd.DataFrame(0, index=np.arange(len(neighborhoods['Name'])),
                        columns=income.columns[1:])
inc.insert(0, 'NEIGHBORHOOD', neighborhoods['Name'])

for i in range(len(inc)):
    neighb = inc['NEIGHBORHOOD'].loc[i]
    temp = props[['WARD_PRECI', neighb]]
    track = []
    for n in range(len(temp)):
        if temp[neighb].loc[n] != 0:
            track.append(temp['WARD_PRECI'].loc[n])
    med = []
    per_c = []
    try:
        for x in track:
            filt = income['WARD_PRECI'] == x
            if float(income[filt]['Median Household Income']) >= 0:
                med.append(float(income[filt]['Median Household Income']))
            if float(income[filt]['Per Capita Income']) >= 0:
                per_c.append(float(income[filt]['Per Capita Income']))
    except TypeError:
        print(income[filt]['Median Household Income'])
    try:
        inc['Median Household Income'].loc[i] = round(sum(med)/len(med))
        inc['Per Capita Income'].loc[i] = round(sum(per_c)/len(per_c))
    except ZeroDivisionError:
        pass

inc = inc.drop(25)
inc

,NEIGHBORHOOD,Median Household Income,Per Capita Income
0,Roslindale,71389,36446
1,Jamaica Plain,67666,39022
2,Mission Hill,35837,28362
3,Longwood,35226,33496
4,Bay Village,81531,72815
5,Leather District,55423,47876
6,Chinatown,68477,60346
7,North End,86551,68660
8,Roxbury,34637,21206
9,South End,72254,59431


In [26]:
income

,WARD_PRECI,Median Household Income,Per Capita Income
0,0113,50182,27780
1,0102,61424,34128
2,0105,59506,30175
3,0203,82648,56539
4,0205,92434,61686
...,...,...,...
250,1818,64901,30285
251,0502A,38433,33295
252,0115,0,7029
253,0104,54235,29589


In [27]:
inc.to_csv('Boston_Neighborhood_Income_Measures_ACS_Data_2011-15.csv')

In [106]:
censusdata.printtable(censusdata.censustable('acs5', 2015, 'B19301'))

Variable     | Table                          | Label                                                    | Type 
-------------------------------------------------------------------------------------------------------------------
B19301_001E  | B19301. Per Capita Income in t | Per capita income in the past 12 months (in 2015 Inflati | int  
-------------------------------------------------------------------------------------------------------------------


In [117]:
censusdata.printtable(censusdata.censustable('acs3', 2012, 'B19013'))
df = censusdata.download('acs1', 2012, censusdata.censusgeo([('state', '25'), ('county', '025'), ('tract', '*')]),
                             ['B19301_001E'])
df

Variable     | Table                          | Label                                                    | Type 
-------------------------------------------------------------------------------------------------------------------
B19013_001E  | B19013.  Median Household Inco | Median household income in the past 12 months (in 2012 i |      
-------------------------------------------------------------------------------------------------------------------


ValueError: Unexpected response (URL: https://api.census.gov/data/2012/acs/acs1?get=NAME,B19301_001E&for=tract:*&in=state:25+county:025): error: unknown/unsupported geography heirarchy 